<a href="https://colab.research.google.com/github/gitkuntal/colab-notebook/blob/master/exp1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install OpenAI package (if not already installed)
%pip install --quiet openai
%pip install openai chromadb
%pip install -U langchain-community
%pip install langchain-openai

In [3]:

# Import required libraries
import getpass
from openai import OpenAI

# Ask for API key securely (you only need to do this once per Colab session)
api_key = getpass.getpass("🔑 Enter your OpenAI API Key: ")

# Initialize client
client = OpenAI(api_key=api_key)

In [25]:

import json
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings


In [28]:
# Load the JSON
with open("search_data.json", "r", encoding="utf-8") as f:
    search_docs = json.load(f)

# Extract texts from search_pages
search_texts = [f"{item['title']} - {item['snippet']}" for item in search_docs['search_pages']]

# Extract texts from pubmed_pages
pubmed_texts = [f"{item['pub_title']} - {item['abstract_snippet']}" for item in search_docs['pubmed_pages']]



# 1. Load doctor_db.json
with open("doctor_db.json", "r", encoding="utf-8") as f:
    doctor_data = json.load(f)

doctors = doctor_data["doctor_db"]

# 2. Convert doctor info into text format for embeddings
doctor_texts = [
    f"Name: {doc['name']}, Specialization: {doc['specialization']}, "
    f"Available Timings: {doc['available_timings']}, Location: {doc['location']}, "
    f"Contact: {doc['contact']}"
    for doc in doctors
]



In [29]:
# 2. Create embedding function wrapper for Chroma
embeddings_client = OpenAIEmbeddings(
    api_key=api_key,
    model="text-embedding-3-small"
)

In [30]:

# Build Chroma collection from search_pages
web_search_db = Chroma.from_texts(
    search_texts,
    embedding=embeddings_client,
    collection_name="web_search_db_demo2"
)
print("✅ Web search database embedded into Chroma successfully!")

# Build another collection from pubmed_pages (optional)
pubmed_db = Chroma.from_texts(
    pubmed_texts,
    embedding=embeddings_client,
    collection_name="pubmed_db_demo2"
)
print("✅ Pubmed database embedded into Chroma successfully!")

# 3. Create Chroma collection with doctor_db
doctor_db = Chroma.from_texts(
    texts=doctor_texts,
    embedding=embeddings_client,
    collection_name="doctor_db_demo2"
)

print("✅ Doctor database embedded into Chroma successfully!")

✅ Web search database embedded into Chroma successfully!
✅ Pubmed database embedded into Chroma successfully!
✅ Doctor database embedded into Chroma successfully!


In [31]:
query = "What are the complications of diabetes?"
results = web_search_db.similarity_search(query, k=2)

for i, res in enumerate(results, 1):
    print(f"{i}. {res.page_content}\n")

1. Diabetes mellitus: The epidemic of the century - Diabetes mellitus is a group of metabolic diseases characterized by chronic hyperglycemia resulting from defects in insulin secretion, insulin action, or both.

2. Diabetes mellitus: The epidemic of the century - Diabetes mellitus is a group of metabolic diseases characterized by chronic hyperglycemia resulting from defects in insulin secretion, insulin action, or both.



In [32]:
query = "What are the complications of diabetes?"
results = pubmed_db.similarity_search(query, k=2)

for i, res in enumerate(results, 1):
    print(f"{i}. {res.page_content}\n")


1. The burden and risks of emerging complications associated with diabetes mellitus - This article discusses evidence for the emergence of a different set of complications associated with diabetes mellitus from the traditional ones. These include increased risks of non‐vascular complications such as cancer, infections, neurodegenerative diseases, and liver disease in patients with long‐standing hyperglycemia.

2. The burden and risks of emerging complications associated with diabetes mellitus - This article discusses evidence for the emergence of a different set of complications associated with diabetes mellitus from the traditional ones. These include increased risks of non‐vascular complications such as cancer, infections, neurodegenerative diseases, and liver disease in patients with long‐standing hyperglycemia.



In [33]:
# 5. Example query
#query = "What are the complications of diabetes?"
query = "Which doctors specialize in diabetology?"
results = doctor_db.similarity_search(query, k=3)

for i, res in enumerate(results, 1):
    print(f"{i}. {res.page_content}\n")

1. Name: Dr. Rajesh Sharma, Specialization: Diabetologist, Available Timings: Tue-Sat, 10 AM - 5 PM, Location: Fortis Hospital, Mumbai, Contact: rajesh.sharma@fortisdoc.in

2. Name: Dr. Rajesh Sharma, Specialization: Diabetologist, Available Timings: Tue-Sat, 10 AM - 5 PM, Location: Fortis Hospital, Mumbai, Contact: rajesh.sharma@fortisdoc.in

3. Name: Dr. Nikhil Verma, Specialization: Diabetologist, Available Timings: Mon-Wed-Fri, 9 AM - 12 PM, Location: Ruby Hall Clinic, Pune, Contact: nikhil.verma@rubyhall.org



In [34]:
from langchain.agents import tool
from langchain.schema import Document

@tool("web_search_tool", return_direct=False)
def web_search_tool(query: str) -> str:
    """Search the Web Search DB for information about diabetes and related topics."""
    results = web_search_db.similarity_search(query, k=3)
    return "\n\n".join([doc.page_content for doc in results])

@tool("pubmed_tool", return_direct=False)
def pubmed_tool(query: str) -> str:
    """Search the PubMed DB for scientific research on diabetes."""
    results = pubmed_db.similarity_search(query, k=3)
    return "\n\n".join([doc.page_content for doc in results])

@tool("doctor_db_tool", return_direct=False)
def doctor_db_tool(query: str) -> str:
    """Search the Doctor DB for information about doctors, specializations, timings, and locations."""
    results = doctor_db.similarity_search(query, k=3)
    return "\n\n".join([doc.page_content for doc in results])


In [36]:
from langchain_openai import ChatOpenAI
from langchain.agents import initialize_agent, AgentType

# Define LLM
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    api_key=api_key
)

# Collect tools
tools = [web_search_tool, pubmed_tool, doctor_db_tool]

# Create agent
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)


In [37]:
response = agent.invoke("Find me research about complications of diabetes and also suggest a doctor in Delhi who specializes in endocrinology.")
print(response)




> Entering new AgentExecutor chain...
I need to gather information on the complications of diabetes through scientific research and also find a specialist doctor in Delhi who focuses on endocrinology. 

Action: pubmed_tool  
Action Input: "complications of diabetes"  
Observation: The burden and risks of emerging complications associated with diabetes mellitus - This article discusses evidence for the emergence of a different set of complications associated with diabetes mellitus from the traditional ones. These include increased risks of non‐vascular complications such as cancer, infections, neurodegenerative diseases, and liver disease in patients with long‐standing hyperglycemia.

The burden and risks of emerging complications associated with diabetes mellitus - This article discusses evidence for the emergence of a different set of complications associated with diabetes mellitus from the traditional ones. These include increased risks of non‐vascular complications such as cancer,

In [38]:
print(response["output"])

Research indicates that diabetes can lead to various complications, including emerging non-vascular issues such as cancer, infections, neurodegenerative diseases, and liver disease, particularly in patients with long-standing hyperglycemia. 

For endocrinology specialists in Delhi, you can consider:
1. Dr. Anjali Mehta - Available Mon-Fri, 9 AM - 2 PM at Apollo Hospital, Delhi. Contact: anjali.mehta@endoclinic.com
2. Dr. Arvind Nair - Available Mon-Fri, 8 AM - 1 PM at AIIMS, New Delhi. Contact: arvind.nair@aiimsmed.edu
